# Using requests python module and NASA API

## 1.1 Requests: HTTP for Humans

*"Requests allows you to send organic, grass-fed HTTP/1.1 requests, without the need for manual labor."*

Let's try to get NASA data using web API

In [ ]:
import requests

## 1.2 APOD (Astronomy Picture of the Day)
For passing parameters to the API using python dictionary

In [ ]:
parameters={'api_key':'<YOUR_KEY>'}
response=requests.get("https://api.nasa.gov/planetary/apod", params=parameters)

In [ ]:
print(response.status_code)

Codes are available as:

In [ ]:
requests.codes.ok

In [ ]:
# get headers of response object
response.headers

In [ ]:
# get text attribute of response object. this is the payload.
# url for picture is included.
response.text

In [ ]:
json_data=response.json()
type(json_data)

In [ ]:
# pip install pillow

In [ ]:
# can get the image on the fly with the following
from PIL import Image
from io import BytesIO

response=requests.get(json_data['url'])
img=Image.open(BytesIO(response.content))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
fig, ax=plt.subplots(figsize=(14, 14))
ax.imshow(img)

# 1.3 Mars rovers photo

In [ ]:
parameters={'api_key':'<YOUR_KEY>'',
           'sol': 1000,
           'camera': 'FHAZ'}
api_end_point='https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos'
response=requests.get(api_end_point, params=parameters)
response.status_code

In [ ]:
json_data=response.json()
# the data is nothing but a dictionary
json_data

In [ ]:
%matplotlib inline
for image_data in json_data['photos']:
    response=requests.get(image_data['img_src'])
    fig, ax=plt.subplots(figsize=(14, 14))
    img=Image.open(BytesIO(response.content))
    ax.imshow(img)

# NASA Techport API

Take a look at the Techport API documentation.

In [ ]:
project_id = 93128
api_end_point = f"https://api.nasa.gov/techport/api/projects/{project_id}"
parameters={'api_key':'<YOUR_KEY>'}

response=requests.get(api_end_point, params=parameters)
response.status_code

In [ ]:
json_data=response.json()
# the data is nothing but a dictionary
json_data['project']['benefits']

# 2 Using REST API for Collecting Tweets with POST and GET
## Note that Twitter has upgraded the API to v2.  

1. Sign up for an account (access level Essential)
2. Creat a new app (this will generate a client key, secret key and bearer token)
3. Review the API documentation

See [here](https://developer.twitter.com/en/docs/api-reference-index) for a complete list of endpoints.  It's a good idea to stick to the Search Tweets endpoints (recent and full-archive) to start.

# 3 POST OAuth2.0/token (for application only authentication)

In [ ]:
# reading the api key and api key secret from a local file
import requests
import json
import base64

with open('twitter_keys.txt') as f:
    lines = f.readlines()
client_key=lines[0][:-1]
client_secret=lines[1][:-1]
# bearer token
bearer_token=lines[2]

In [ ]:
# Note that we'll be making App only requests which only require the bearer token.
# generating bearer_token...note that we can just get the bearer token from the web interface as well.

# first convert the str to a bytes-like object
key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
# encode with base64
b64_encoded_key = base64.b64encode(key_secret)
# get string representation of the base64 encoding.
b64_encoded_key = b64_encoded_key.decode('ascii')

auth_endpoint='https://api.twitter.com/oauth2/token'

auth_headers = {'Authorization': 'Basic {}'.format(b64_encoded_key),
               'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}
auth_data = {'grant_type': 'client_credentials'}

response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)

#print(response.status_code)
#print(response.text)

json_data = response.json()
bearer_token = json_data['access_token']

# Asynch Ex: Let's Run Some Queries Against the Twitter API

#### We need to build our query
[See here](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query) for information on building queries to search Tweets.

The documentation on [building high-level queries](https://developer.twitter.com/en/docs/tutorials/building-high-quality-filters) is also helpful.

The search endpoints accept a single query with a GET request and return a set of historical Tweets that match the query.  Queries are made up of operators that are used to match on a variety of Tweet attributes. 

__Operators__ are used to match tweets.  You can find a table of operators [here](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#list).

There are two types of operators. Standalone and conjunction-required.

__Standalone__ can be used alone or with any other operators.  For example, the #hashtag operator is standalone, so this is a valid query: #bananastand

__Conjunction-required operators__ cannont be used alone in a query.  They can be used when at least one standalone operator is included in the query....using conjunction-required alone would be too general.

The response fields for recent Tweet search can be found [here](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent)

We can also use boolean operators and grouping to string together multiple operators in a single query. We have access to AND, OR, NOT.  Prepend a dash (-) to a keyword or operator to negate it. For example -hotdog or -is:retweet. 

We can group with parentheses. For example, (hot dog) OR (#hotdog has:images) will return either Tweets containing the words hot and dog or Tweets with images that have the hashtag #hotdog.  This is not an XOR.

Feel free to use the Twitter [query builder](https://developer.twitter.com/apitools/query?query=) when crafting your queries.  Actually...definitely use it.  This tool will help you get comfortable with the query syntax and will validate queries for you.

## Let's run a very simple query

In [ ]:
from pprint import pprint

In [ ]:
# Example 1

# specify the endpoint
endpoint = "https://api.twitter.com/2/tweets/search/recent"

# create header object with bearer_token
headers = {'Authorization': 'Bearer {}'.format(bearer_token)}

# specify query 
query = "John Cheever"

# specify tweet_fields
tweet_fields = 'tweet.fields=text'

# specify max results (optional)
max_results = 'max_results=10'

# create the url for request
url = endpoint + "?query=" + query + "&" + tweet_fields + "&" + max_results

print(url)
# make the request
response = requests.request("GET", url, headers=headers)

print(response.status_code)

#data = response.json()
#pprint.pprint(data)

In [ ]:
data = response.json()
pprint(data)

#### And now a slightly more complex query....

In [ ]:
# specify query
query = '"climate change" is:verified -is:retweet lang:en'

# specify field for payload
tweet_fields = "tweet.fields=text,id,author_id,created_at"

# specify max results (optional)
max_results = 'max_results=10'

# get expanded author_id object
#expansions = "expansions=author_id"

# create the url for request
url = endpoint + "?query=" + query + "&" + tweet_fields + "&" + max_results

# make the request
response = requests.request("GET", url, headers=headers)

print(response.status_code)

cc_data = response.json()
pprint(cc_data)

# EX 1: Modularizing our Approach
Now we'll modularize our code a bit by building a function to execute our request.

In [ ]:
def search_twitter(query, tweet_fields, max_results=10, bearer_token=bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}

    url = "https://api.twitter.com/2/tweets/search/recent?query={}&{}&max_results={}".format(
        query, tweet_fields, max_results
    )
    response = requests.request("GET", url, headers=headers)

    print(response.status_code)

    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

#### Let's look for a single term.  Note the quoting.  I want to find instances of "pickle ball" not instances of "pickle" and "ball".

In [ ]:
query = '"pickle ball"'
tweet_fields = 'tweet.fields=text,author_id,created_at'
max_results = 10

json_response=search_twitter(query=query,
                             tweet_fields=tweet_fields,
                             max_results=max_results,
                             bearer_token=bearer_token)

pprint(json_response)

#### Now a more complex query...

In [ ]:
# Example 2
query = '"stranger things" has:mentions -is:retweet (has:media OR has:links)'
tweet_fields = 'tweet.fields=text,author_id,created_at'

json_response=search_twitter(query=query,tweet_fields=tweet_fields,bearer_token=bearer_token)
pprint(json_response)

# Ex2 More Sophisticated Query

We've mostly focused on the query, but the request we submit has other paramters that can be specified as well.  We've already used some of them (max_results, tweet_fields).  Each endpoint will detail the parameters we can specify.  For recent tweet search only the query is a required parameter.

In [ ]:
def connect_to_endpoint(url, headers, params):
    
    response = requests.get(url, headers=headers, params=params)
    
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [ ]:
url = "https://api.twitter.com/2/tweets/search/recent"

headers = {"Authorization": "Bearer {}".format(bearer_token)} 

query_params = {'query': '"Russell Wilson" "Hackett" has:mentions -is:retweet has:media is:verified has:images',
                'max_results': 10,
                'tweet.fields': 'id,author_id,text,geo,conversation_id',
                'expansions': 'author_id,geo.place_id',
                'user.fields': 'name,username,verified,location',
                'place.fields': 'country_code,geo,name,place_type'}

In [ ]:
data = connect_to_endpoint(url, headers, query_params)

In [ ]:
pprint(data)